In [1]:
!pip install pmdarima

In [2]:
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error as mae
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import datetime
from sklearn import svm
import warnings
warnings.filterwarnings("ignore")

# Import the necessary libraries
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
#import my_heat_wave_prediction_function # Replace with the name of your heat wave prediction function

def predict():
    prediction_label.config(text='Please wait....')
    dist=district_var.get().lower()
    df=pd.read_csv(dist+".csv")
    newdf=pd.read_csv(dist+"2023.csv")
    combineddf=df.append(newdf)
    features=df.iloc[:,2:]
    for f in df.corr()['Max Temp'].keys():
        if abs(df.corr()['Max Temp'][f])<0.20:
            df=df.drop([f],axis=1)
            if f=='AQI':
                continue
            combineddf=combineddf.drop([f],axis=1)
    train=combineddf[:len(df)]
    x_train=train.drop(['Date','Max Temp','AQI'],axis=1)
    y_train=train[['Mandal','Max Temp']]
    test=combineddf[len(df):]
    x_test=test.drop(['Date','Max Temp','AQI'],axis=1)
    def rfr(mandal):
        reg=RandomForestRegressor(n_estimators=100)
        x_mandal_train=x_train[x_train['Mandal']==mandal]
        x_mandal_train=x_mandal_train.drop(['Mandal'],axis=1)
        y_mandal_train=y_train[y_train['Mandal']==mandal]
        y_mandal_train=y_mandal_train.drop(['Mandal'],axis=1)
        x_mandal_test=x_test[x_test['Mandal']==mandal]
        x_mandal_test=x_mandal_test.drop(['Mandal'],axis=1)
        check=True
        if len(x_mandal_train)==0 or len(x_mandal_test)==0:
            return
        else:
            reg.fit(x_mandal_train,y_mandal_train)
            y_predict=reg.predict(x_mandal_test)
            count=0
            for i,row in newdf[newdf['Mandal']==mandal].iterrows():
                newdf.at[i,'Max Temp']=y_predict[count]
                if dist=='warangal':
                    if count==55 and check:
                        count=0
                        check=False
                        
                count+=1
    for m in combineddf['Mandal'].unique():
        rfr(m)
    df=pd.read_csv(dist+".csv")
    combineddf=df.append(newdf)
    features=df.iloc[:,2:]
    for f in df.corr()['AQI'].keys():
        if abs(df.corr()['AQI'][f])<0.25:
            df=df.drop([f],axis=1)
            combineddf=combineddf.drop([f],axis=1)
    train=combineddf[:len(df)]
    x_train=train.drop(['Date','AQI'],axis=1)
    y_train=train[['Mandal','AQI']]
    test=combineddf[len(df):]
    x_test=test.drop(['Date','AQI'],axis=1)
    def sv(mandal):
        reg=svm.SVR(C=1.0, kernel='poly')
        x_mandal_train=x_train[x_train['Mandal']==mandal]
        x_mandal_train=x_mandal_train.drop(['Mandal'],axis=1)
        y_mandal_train=y_train[y_train['Mandal']==mandal]
        y_mandal_train=y_mandal_train.drop(['Mandal'],axis=1)
        x_mandal_test=x_test[x_test['Mandal']==mandal]
        x_mandal_test=x_mandal_test.drop(['Mandal'],axis=1)
        if len(x_mandal_train)==0 or len(x_mandal_test)==0:
            return
        else:
            reg.fit(x_mandal_train,y_mandal_train)
            y_predict=reg.predict(x_mandal_test)
            count=0
            for i,row in newdf[newdf['Mandal']==mandal].iterrows():
                newdf.at[i,'AQI']=y_predict[count]
                count+=1

    

    for m in combineddf['Mandal'].unique():
        sv(m)

    month=0
    mon=month_var.get()
    prediction='No of heatwave days in '+mon+' :'
    if(mon=='January'):
        month='01'
    elif(mon=='February'):
        month='02'
    elif(mon=='March'):
        month='03'
    elif(mon=='April'):
        month='04'
    elif(mon=='May'):
        month='05'
    elif(mon=='June'):
        month='06'
    elif(mon=='July'):
        month='07'
    elif(mon=='August'):
        month='08'
    elif(mon=='September'):
        month='09'
    elif(mon=='October'):
        month='10'
    elif(mon=='November'):
        month='11'
    else:
        month='12'
    
    def printHeatwave(month,prediction):
        # read the CSV file into a Pandas DataFrame
        data = newdf.copy()
        data=data[data['Date'].str.contains('2023')]

        # convert the date column to a pandas datetime object
        data['Date'] = pd.to_datetime(data['Date'])

        # filter the data to include only the rows with temperature above 45 degrees Celsius
        data_above_45 = data.loc[data['Max Temp'] >= 43]

        # group the data by date and create a dictionary mapping dates to a list of mandals
        date_mandal_dict = {}
        grouped_data = data_above_45.groupby('Date')
        for date, group in grouped_data:
            mandals_above_45 = list(group['Mandal'].unique())
            date_mandal_dict[date.date()] = mandals_above_45

        # sort the keys of the dictionary
        sorted_keys = sorted(date_mandal_dict.keys())
        sorted_dict = dict(sorted(date_mandal_dict.items(), key=lambda x: x[0]))

        my_dict={}

        for key in sorted_dict:
            dt = datetime.datetime(key.year, key.month, 1)
            # format the datetime object as a string in the format "YYYY-MM"
            key1 = dt.strftime('%Y-%m')
            # add the formatted string as a key to the dictionary with an empty list as the value
            my_dict[key1] = sorted_dict[key]


        n= len(sorted_keys)

        year_dict = {}
        satyam_dict = {}
        mandal_dict={}
        for i in range(0,n):
            if len(date_mandal_dict[sorted_keys[i]]) >=2:
                dt = datetime.datetime(sorted_keys[i].year, sorted_keys[i].month, 1)
                key1 = dt.strftime('%m')
                if key1 in satyam_dict:
                    if not satyam_dict[key1]:
                        satyam_dict[key1] = 1
                        mandal_dict[key1]=set()
                        for m in date_mandal_dict[sorted_keys[i]]:
                            mandal_dict[key1].add(m)
                    else:
                        satyam_dict[key1]+=1
                        for m in date_mandal_dict[sorted_keys[i]]:
                            mandal_dict[key1].add(m)

                else:
                    mandal_dict[key1]=set()
                    satyam_dict[key1] = 1
                    for m in date_mandal_dict[sorted_keys[i]]:
                            mandal_dict[key1].add(m)

                if sorted_keys[i].year in year_dict:
                    if not year_dict[sorted_keys[i].year]:
                        year_dict[sorted_keys[i].year] = 1
                    else:
                        year_dict[sorted_keys[i].year] += 1
                else:
                    year_dict[sorted_keys[i].year]=1;
        if not month in satyam_dict:
            prediction+='0'
        else:
            prediction+=str(satyam_dict[month])
            prediction+='\n'
            prediction+='Mandals affected :'
            prediction+=str(mandal_dict[month])
        return prediction
    
    prediction=printHeatwave(month,prediction)
    prediction+='\n'
    def printAqi(month,prediction):
        data=newdf[newdf['Date'].str.contains(month,'-2023')]
        sum=0
        range =0
        for aqi in data['AQI']:
            sum+=aqi
        avg=sum/len(data)
        range=avg*0.05
        prediction+="Average AQI for the month "
        prediction+=mon
        prediction+=" : "
        prediction+=str(round(avg,2))
        prediction+="+-"
        prediction+=str(round(range,2))
        return prediction
    prediction=printAqi(month,prediction)
    prediction_label.config(text=prediction)

# Create the main application window
root = tk.Tk()
root.title("Heat Wave Prediction")
root.geometry("900x500") # Set the size of the window
root.configure(bg="#E1F8DC") # Set the background color

# Add a heading to the application window
heading_label = ttk.Label(root, text="AQI and Heat Wave Prediction", font=("Georgia", 20, "bold"), foreground="#000000", background="#E1F8DC")
heading_label.place(relx=0.5, rely=0.03, anchor="center")


# Create a label and a dropdown menu for the district
district_label = ttk.Label(root, text="Please Select a District:", font=("Georgia", 12), foreground="#000000", background="#E1F8DC")
district_label.pack(side="top", padx=20, pady=50)
district_var = tk.StringVar()
#district_var.set("---Select District---")
district_dropdown = ttk.Combobox(root, textvariable=district_var,font=("Georgia", 12))
district_dropdown.pack(side="top", padx=10, pady=5)
district_dropdown['values'] = ("Adilabad", "Nizamabad", "Karimnagar","Khammam","Warangal") # Replace with your own list of districts

# Create a label and a dropdown menu for the month
month_label = ttk.Label(root, text="Choose a month:", font=("Georgia", 12), foreground="#000000", background="#E1F8DC")
month_label.pack(side="top", padx=10, pady=20)
month_var = tk.StringVar()
#month_var.set("---Select Month---")
month_dropdown = ttk.Combobox(root, textvariable=month_var, font=("Georgia", 12))
month_dropdown.pack(side="top", padx=20, pady=5)
month_dropdown['values'] = ("January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December") # Replace with your own list of months

# Create a button to trigger the heat wave prediction function
predict_button = ttk.Button(root, text="Predict",command=predict, style="my.TButton")
predict_button.pack(side="top", padx=20, pady=20)

# Create a label to display the predicted output
prediction_label = ttk.Label(root, text="", font=("Georgia", 14), foreground="#000000",background="#FFFFFF")
prediction_label.pack(side="top", padx=20, pady=20)

# Define a custom style for the button
style = ttk.Style()
style.configure("my.TButton", font=("Georgia", 12), foreground="#000000", background="#FFFFFF")

# Run the main event loop
root.mainloop()

